ConvLSTM trained on gridded forcings for all stations

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190723-135859'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

CUDA Available
2019-07-23 13:58:59,797 - 20190723-135859 - cuda devices: ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']


In [4]:
seq_len = 50
seq_steps = 1

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)  # first day for which to make a prediction in train set
train_end = '2012-09-30'
val_start = '2012-10-01'
val_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [5]:
rdrs_vars = [4,5]
train_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end)
val_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, val_start, val_end, conv_scalers=train_dataset.conv_scalers)
test_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, conv_scalers=train_dataset.conv_scalers)

../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]


In [6]:
class ConvLSTMGrid(nn.Module):
    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers, dropout=0.0):
        super(ConvLSTMGrid, self).__init__()
        self.conv_lstm = conv_lstm.ConvLSTM((input_size[0], input_size[1]), input_dim, hidden_dim, kernel_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout2d(p=dropout)
        self.conv_out = nn.Conv2d(hidden_dim[-1], 1, (1,1))
        
    def forward(self, input_tensor, hidden_state=None):
        convlstm_out, hidden = self.conv_lstm(input_tensor, hidden_state=hidden_state)
        convlstm_out = self.dropout(convlstm_out[-1][:,-1,:,:,:])  # last output of last layer
        return self.conv_out(convlstm_out)[:,0,:,:], hidden

In [7]:
# Train model
num_epochs = 100
learning_rate = 2e-3
patience = 50
min_improvement = 0.05
best_loss_model = (-1, np.inf, None)

# Prepare model
stateful_lstm = True
num_conv_layers = 3
conv_hidden_dims = [8] * num_conv_layers
batch_size = 16
kernel_size = [(3,3), (3,3), (3,3)]
dropout = 0.3

model = ConvLSTMGrid((train_dataset.conv_height, train_dataset.conv_width), train_dataset.n_conv_vars, conv_hidden_dims, kernel_size, num_conv_layers, dropout=dropout).to(device)
model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

writer = SummaryWriter()
param_description = {'time_stamp': time_stamp, 'H_conv': conv_hidden_dims, 'batch_size': batch_size, 'num_conv_layers': num_conv_layers, 'kernel_size': kernel_size, 'loss': loss_fn, 
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'stateful_lstm': stateful_lstm, 'dropout': dropout,
                     'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'val_start': val_start, 
                     'val_end': val_end, 'test_start': test_start, 'test_end': test_end, 'n_conv_vars': train_dataset.n_conv_vars, 'model': str(model).replace('\n','').replace(' ', ''),
                     'train len':len(train_dataset), 'val len': len(val_dataset), 'conv_height': train_dataset.conv_height, 'conv_width': train_dataset.conv_width, 'test len': len(test_dataset)}
writer.add_text('Parameter Description', str(param_description))

In [8]:
if stateful_lstm:
    train_sampler = datasets.StatefulBatchSampler(train_dataset, batch_size)
    val_sampler = datasets.StatefulBatchSampler(val_dataset, batch_size)
    test_sampler = datasets.StatefulBatchSampler(test_dataset, batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_sampler, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler, pin_memory=True)
else:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=True)

In [9]:
for epoch in range(num_epochs):
    model.train()

    epoch_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, train_batch in enumerate(train_dataloader):
        y_train = train_batch['y'].reshape(-1).to(device, non_blocking=True)
        mask = train_batch['mask'].reshape(-1).to(device, non_blocking=True)
        if mask.sum() == 0:
            print('Batch {} has no target values. skipping.'.format(i))
            continue
        if not stateful_lstm:
            conv_hidden_states = None
        
        y_pred, conv_hidden_states = model(train_batch['x_conv'].to(device), hidden_state=conv_hidden_states)
        y_pred = y_pred.reshape(-1) * mask  # ignore grid cells that have no target value
        loss = loss_fn(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_losses += (loss * y_train.shape[0] / mask.sum()).detach()  # only average over losses where we had a target
        
    epoch_loss = (epoch_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(epoch_loss))
    writer.add_scalar('loss', epoch_loss, epoch)
    
    # eval on validation split
    model.eval()
    val_losses = torch.tensor(0.0)
    for i, val_batch in enumerate(val_dataloader):
        y_val = val_batch['y'].reshape(-1).to(device, non_blocking=True)
        mask = val_batch['mask'].reshape(-1).to(device, non_blocking=True)
        if not stateful_lstm:
            conv_hidden_states = None
        
        batch_pred, conv_hidden_states = model(val_batch['x_conv'], hidden_state=conv_hidden_states)
        batch_pred = batch_pred.detach().reshape(-1)
        val_losses += (loss_fn(batch_pred * mask, y_val).detach() * y_val.shape[0] / mask.sum())
        
    val_mse = (val_losses / len(val_dataloader)).item()
    print('Epoch {} mean val mse:    \t{}'.format(epoch, val_mse))
    writer.add_scalar('loss_eval', val_mse, epoch)

    if val_mse < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_mse, model.state_dict())  # new best model
        load_data.pickle_model('ConvLSTM_VIC', model, 'allStations', time_stamp)
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break

print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.pickle_model('ConvLSTM_VIC', model, 'allStations', time_stamp)

Epoch 0 mean train loss:	3978.2626953125
Epoch 0 mean val mse:    	730.3317260742188
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl


/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ConvLSTMGrid. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean train loss:	3933.3056640625
Epoch 1 mean val mse:    	715.0436401367188
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 2 mean train loss:	3903.998291015625
Epoch 2 mean val mse:    	703.87060546875
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 3 mean train loss:	3880.177001953125
Epoch 3 mean val mse:    	695.4714965820312
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 4 mean train loss:	3858.295166015625
Epoch 4 mean val mse:    	689.1996459960938
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 5 mean train loss:	3844.72412109375
Epoch 5 mean val mse:    	684.91845703125
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 6 mean train loss:	3824.968017578125
Epoch 6 mean val mse:    	682.1114501953125
Saved model as ../pickle/models/ConvLSTM_VIC_allStations_20190723-135859.pkl
Epoch 7 mean train loss

In [10]:
logger.warning('predicting')
model.eval()

predictions = []
for i, test_batch in enumerate(test_dataloader):
    if not stateful_lstm:
        conv_hidden_states = None
        
    pred, conv_hidden_states = model(test_batch['x_conv'], hidden_state=conv_hidden_states)
    predictions.append(pred.detach())
    
predictions = torch.cat(predictions).cpu()

if stateful_lstm:
    # reorder time series
    pred_indices = np.array(list(test_sampler.__iter__())).reshape(-1)
    predictions = predictions[pred_indices.argsort()]

2019-07-23 14:14:41,856 - 20190723-135859 - predicting


In [11]:
actuals = test_dataset.data_runoff.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_list = []
mse_list = []
predictions_df = pd.DataFrame(columns=actuals.columns)
for station in actuals['station'].unique():
    row, col = test_dataset.station_to_row_col[station]
    
    act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != act.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for station {}. Ignoring excess actuals.'.format(len(predictions), len(act), station))
        act = act.iloc[:predictions.shape[0]]
    pred = pd.DataFrame({'runoff': predictions[:,row,col]}, index=act.index)
    pred['station'] = station
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    
    nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))
writer.add_scalar('nse_median', np.median(nse_list))

02GA047 	NSE: -0.3194422952725249 	MSE: 104.76698466261509 (clipped to 0)


/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


04213000 	NSE: -0.2244287599263053 	MSE: 222.50463813536334 (clipped to 0)
04176500 	NSE: -0.02765795678694949 	MSE: 710.6856258101308 (clipped to 0)
02GG003 	NSE: -0.002425442678314793 	MSE: 484.0550405571684 (clipped to 0)
04214500 	NSE: -0.7374970885130614 	MSE: 212.11593562968693 (clipped to 0)
02GC026 	NSE: -0.14104242708311632 	MSE: 185.9024387623806 (clipped to 0)
04174500 	NSE: -0.0013429235378223936 	MSE: 95.95168039079141 (clipped to 0)
02GG013 	NSE: -4.5052848941601065 	MSE: 211.77946299488968 (clipped to 0)
04161820 	NSE: -1.764027424008273 	MSE: 88.16509870358028 (clipped to 0)
04159492 	NSE: -0.10093016017662948 	MSE: 511.47341873894663 (clipped to 0)
04200500 	NSE: -0.003946047577142453 	MSE: 1017.6804096951757 (clipped to 0)
02GB001 	NSE: -0.6505129257700737 	MSE: 12393.927902890488 (clipped to 0)
04208504 	NSE: -0.5477858990490754 	MSE: 1528.6543798128957 (clipped to 0)
04199000 	NSE: -0.011670898765089976 	MSE: 1047.5529556722083 (clipped to 0)
02GG002 	NSE: -0.160853

In [12]:
writer.close()

In [13]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), actuals.rename({'runoff': 'actual'}, axis=1), 
                   on=['date', 'station'])[['date', 'station', 'prediction', 'actual']]
load_data.pickle_results('ConvLSTM_VIC', save_df, time_stamp)

'ConvLSTM_VIC_20190723-135859.pkl'

In [14]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190723-141453'